In [1]:
print("Hello, world")

Hello, world


In [2]:
import pandas as pd
import numpy as np
import faiss

from tqdm.notebook import tqdm
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler

## Head-on Solution

In [3]:
df_base = pd.read_parquet("../tmp/cleared_df_base.parquet")

In [4]:
df_base.head()

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083893,11.152912,-64.426758,-118.880890,216.482437,-104.698059,-469.070587,44.348083,120.915344,181.449707,...,-138.777634,-42.808693,38.800827,-151.762177,-74.389091,63.666340,-4.703861,92.933609,115.269188,-60.830353
1-base,-34.562202,13.332763,-69.787613,-166.533478,57.680607,-86.098373,-85.076668,-35.637436,119.718636,195.234192,...,-148.571655,-117.767525,41.099998,-157.829407,-94.446808,68.202110,24.346846,179.937927,116.834000,-59.524609
2-base,-54.233746,6.379371,-29.210136,-133.413834,150.895828,-99.435326,52.554794,62.381706,128.951447,164.381470,...,-134.658218,-76.397797,46.011803,-207.144424,127.325569,65.566177,66.325684,81.073486,116.594154,-32.527206
3-base,-87.520126,4.037884,-87.803032,-185.067627,76.369537,-58.985165,-383.182831,-33.611237,122.031914,136.233582,...,-149.326691,-70.647942,-6.358921,-147.201050,-37.692749,66.202888,-20.566910,137.206940,117.474098,-72.915489
4-base,-72.743851,6.522048,43.671265,-140.608032,5.820023,-112.074081,-397.711273,45.182499,122.167183,112.119064,...,-125.317329,-57.199104,56.642403,-159.351837,85.944725,66.766319,-2.505783,65.315285,135.051590,0.319401


In [5]:
df_train = pd.read_parquet("../tmp/cleared_df_train.parquet")
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.936676,187.517487,-87.144928,-347.360596,38.307602,109.085564,30.413513,...,-105.427818,70.107361,-155.802567,-101.965942,65.903793,34.457500,62.642094,134.763596,-25.958572,675816-base
1-query,-87.776367,6.806268,-32.054546,-177.260391,120.803329,-83.810593,-94.572746,-78.433090,124.915901,140.331070,...,-40.812233,4.669178,-151.697708,-1.638704,68.170876,25.096191,89.974976,130.589630,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118591,-180.401978,190.128433,-50.837620,26.943937,-30.447489,125.771164,211.607819,...,-52.031662,78.039764,-169.146194,82.144188,66.008217,18.400496,212.409729,121.931473,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.011360,94.652840,-109.255409,-775.150146,79.186523,124.003098,242.650650,...,-61.567047,44.515266,-145.416748,93.990982,64.131348,106.061920,83.178757,118.277725,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.052536,57.127068,-16.239529,-321.317963,45.984676,125.941284,103.392670,...,-68.770599,45.028912,-196.092072,-117.626335,66.926224,42.456169,77.621765,92.479927,-21.149351,717819-base


In [ ]:
dims = df_base.shape[1]
n_cells = 50
# how many cells
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))

base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
#del df_base

headon_solution_df_train = df_train.copy()
targets = headon_solution_df_train["Target"]
headon_solution_df_train.drop("Target", axis=1, inplace=True)

# %%time
k=5
vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)

acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))

### Вывод
Решение в лоб - 5.575 попугаев.
Очень низкая оценка.

## Scalers

### StandardScaler

In [11]:
def checkStandardScaler(base, train):
    scaler = StandardScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [12]:
sscaler_base, sscaler_train, _ = checkStandardScaler(df_base, df_train)

In [13]:
def buildA5Score(df_base, df_train, short=True):
    dims = df_base.shape[1]
    n_cells = 50
    # how many cells
    quantizer = faiss.IndexFlatL2(dims)
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=50
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] for r in el])
    
    return 100 * acc / len(idx)

In [14]:
%%time
print("A@5 Score on Standard Scaler :", buildA5Score(sscaler_base, sscaler_train))

A@5 Score on Standard Scaler : 64.654
CPU times: total: 11min 36s
Wall time: 33.9 s


### RobustScaler

In [15]:
def checkRobustScaler(base, train):
    scaler = RobustScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [16]:
rscaler_base, rscaler_train, _ = checkRobustScaler(df_base, df_train)

In [17]:
%%time
print("A@5 Score on Robust Scaler :", buildA5Score(rscaler_base, rscaler_train))

A@5 Score on Robust Scaler : 66.507
CPU times: total: 11min 55s
Wall time: 34.6 s


### MinMaxScaler

In [18]:
def checkMinMaxScaler(base, train):
    scaler = MinMaxScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [19]:
mmscaler_base, mmscaler_train, _ = checkMinMaxScaler(df_base, df_train)

In [20]:
%%time
print("A@5 Score on Min Max Scaler :", buildA5Score(mmscaler_base, mmscaler_train))

A@5 Score on Min Max Scaler : 10.776
CPU times: total: 11min 52s
Wall time: 34.5 s


### Normalizer

In [21]:
def checkNormalizer(base, train):
    scaler = Normalizer()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [22]:
normscaler_base, normscaler_train, _ = checkNormalizer(df_base, df_train)

In [23]:
%%time
print("A@5 Score on Normalizer :", buildA5Score(normscaler_base, normscaler_train))

A@5 Score on Normalizer : 6.554
CPU times: total: 11min 2s
Wall time: 32.3 s


### MaxAbsScaler

In [24]:
def checkMaxAbsScaler(base, train):
    scaler = MaxAbsScaler()
    scaler.fit(base)

    tbase = scaler.transform(base)
    tbase = pd.DataFrame(tbase, columns=base.columns)
    tbase["Id"] = base.index
    tbase = tbase.set_index(["Id"])
    
    ttrain = scaler.transform(train[base.columns])

    ttrain = pd.DataFrame(ttrain, columns=base.columns)
    ttrain["Id"] = train.index
    ttrain = ttrain.set_index(["Id"])
    ttrain["Target"] = train["Target"]

    return tbase, ttrain, scaler

In [25]:
maxAbsscaler_base, maxAbsscaler_train, _ = checkMaxAbsScaler(df_base, df_train)

In [26]:
%%time
print("A@5 Score on Max Abs Scaler :", buildA5Score(maxAbsscaler_base, maxAbsscaler_train))

A@5 Score on Max Abs Scaler : 15.184
CPU times: total: 12min 17s
Wall time: 35.7 s


## FIASS Tuning

In [27]:
def buildA5ScoreV2(df_base, df_train, short=True, k_value=50):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    #quantizer = faiss.IndexHNSWFlat(dims)
    #idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    idx_l2 = faiss.IndexLSH(dims, 1024)
    if short:
        idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    else:
        idx_l2.train(np.ascontiguousarray(df_base.values).astype('float32'))
        idx_l2.add(np.ascontiguousarray(df_base.values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=k_value
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] for r in el])
    
    return 100 * acc / len(idx)

In [28]:
rscaler_base, rscaler_train, _ = checkRobustScaler(df_base, df_train)

In [29]:
%%time
print("A@5 Score on Robust Scaler V2:", buildA5ScoreV2(rscaler_base, rscaler_train))

A@5 Score on Robust Scaler V2: 5.185
CPU times: total: 58.6 s
Wall time: 3.38 s


### IndexHNSWFlat

In [30]:
def buildA5ScoreV3(df_base, df_train):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    #quantizer = faiss.IndexFlatL2(dims)
    #idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)
    #idx_l2 = faiss.IndexLSH(dims, 1024)
    #M = 2
    #d2 = int((dims + M - 1) / M) * M
    #q = faiss.IndexPQ (d2, M, 4)
    #index_pq = faiss.IndexRefineFlat(q)
    #remapper = faiss.RemapDimensionsTransform (dims, d2, True)
    #idx_l2 = faiss.IndexPreTransform (remapper, index_pq)
    idx_l2 = faiss.index_factory(dims, "OPQ16_64,IMI2x8,PQ8+16")
    
    idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=250
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] if r in base_index else 0 for r in el])
    
    return 100 * acc / len(idx)



In [31]:
%%time
print("A@5 Score on Robust Scaler V3:", buildA5ScoreV3(rscaler_base, rscaler_train))

A@5 Score on Robust Scaler V3: 3.653
CPU times: total: 19min 9s
Wall time: 51.4 s


In [32]:
def buildA5ScoreV4(df_base, df_train):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    quantizer = faiss.IndexFlatL2(dims)
    idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

    idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=250
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] if r in base_index else 0 for r in el])
    
    return 100 * acc / len(idx)
    

In [33]:
%%time
print("A@5 Score on Robust Scaler V3:", buildA5ScoreV4(rscaler_base, rscaler_train))

A@5 Score on Robust Scaler V3: 4.368
CPU times: total: 12.4 s
Wall time: 3.01 s


In [34]:
def buildA5ScoreV5(df_base, df_train):
    dims = df_base.shape[1]
    n_cells = 100
    # how many cells
    idx_l2 = faiss.index_factory(dims, "IVF69,SQ8")

    idx_l2.train(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    idx_l2.add(np.ascontiguousarray(df_base[:50000].values).astype('float32'))
    
    base_index = {k: v for k, v in enumerate(df_base.index.to_list())}
    #del df_base
    
    headon_solution_df_train = df_train.copy()
    targets = headon_solution_df_train["Target"]
    headon_solution_df_train.drop("Target", axis=1, inplace=True)

    k=250
    vecs, idx = idx_l2.search(np.ascontiguousarray(headon_solution_df_train.values).astype('float32'), k)
    acc = 0
    for target, el in zip(targets.values.tolist(), idx.tolist()):
        acc += int(target in [base_index[r] if r in base_index else 0 for r in el])
    
    return 100 * acc / len(idx)
    

In [35]:
%%time
print("A@5 Score on Robust Scaler V3:", buildA5ScoreV5(rscaler_base, rscaler_train))

A@5 Score on Robust Scaler V3: 4.377
CPU times: total: 20.2 s
Wall time: 3.37 s


## Feature checking

In [36]:
extra_columns = []

In [37]:
%%time
max_score = 0;
max_score_field = "";

original_column_list = rscaler_base.columns.copy().to_list()
for i in original_column_list:
    temp_columns = original_column_list.copy()
    temp_columns.remove(i)
    train_columns = temp_columns.copy()
    train_columns.append("Target")
    score = buildA5ScoreV2(rscaler_base[temp_columns], rscaler_train[train_columns])
    print(f"A@5 without {i}:", score)
    if score > max_score:
        max_score = score
        max_score_field = i

A@5 without 0: 5.177
A@5 without 1: 5.173
A@5 without 2: 5.164
A@5 without 3: 5.174
A@5 without 4: 5.171
A@5 without 5: 5.17
A@5 without 6: 5.169
A@5 without 7: 5.177
A@5 without 8: 5.177
A@5 without 9: 5.18
A@5 without 10: 5.177
A@5 without 11: 5.174
A@5 without 12: 5.175
A@5 without 13: 5.178
A@5 without 14: 5.177
A@5 without 15: 5.173
A@5 without 16: 5.174
A@5 without 17: 5.178
A@5 without 18: 5.178
A@5 without 19: 5.179
A@5 without 20: 5.177
A@5 without 22: 5.167
A@5 without 23: 5.173
A@5 without 24: 5.174
A@5 without 26: 5.176
A@5 without 27: 5.173
A@5 without 28: 5.171
A@5 without 29: 5.171
A@5 without 30: 5.168
A@5 without 31: 5.176
A@5 without 32: 5.178
A@5 without 33: 5.173
A@5 without 34: 5.176
A@5 without 35: 5.175
A@5 without 36: 5.171
A@5 without 37: 5.174
A@5 without 38: 5.177
A@5 without 39: 5.174
A@5 without 40: 5.17
A@5 without 41: 5.179
A@5 without 42: 5.175
A@5 without 43: 5.169
A@5 without 44: 5.167
A@5 without 45: 5.161
A@5 without 46: 5.166
A@5 without 47: 5.174
A

In [38]:
extra_columns.append(max_score_field)
print(f"Max score on {max_score}, {max_score_field}")

Max score on 5.191, 57


In [39]:
%%time
max_score = 0;
max_score_field = "";

original_column_list = rscaler_base.columns.copy().to_list()
for i in original_column_list:
    if i not in extra_columns:
        temp_columns = original_column_list.copy()
        temp_columns.remove(i)

        for k in extra_columns:
            if k in temp_columns:
                temp_columns.remove(k)

        
        train_columns = temp_columns.copy()
        train_columns.append("Target")
        score = buildA5ScoreV2(rscaler_base[temp_columns], rscaler_train[train_columns])
        print(f"A@5 without {i}:", score)
    if score > max_score:
        max_score = score
        max_score_field = i

A@5 without 0: 5.178
A@5 without 1: 5.175
A@5 without 2: 5.168
A@5 without 3: 5.176
A@5 without 4: 5.172
A@5 without 5: 5.178
A@5 without 6: 5.168
A@5 without 7: 5.18
A@5 without 8: 5.171
A@5 without 9: 5.176
A@5 without 10: 5.168
A@5 without 11: 5.17
A@5 without 12: 5.167
A@5 without 13: 5.167
A@5 without 14: 5.169
A@5 without 15: 5.172
A@5 without 16: 5.176
A@5 without 17: 5.181
A@5 without 18: 5.18
A@5 without 19: 5.18
A@5 without 20: 5.186
A@5 without 22: 5.18
A@5 without 23: 5.179
A@5 without 24: 5.188
A@5 without 26: 5.188
A@5 without 27: 5.182
A@5 without 28: 5.179
A@5 without 29: 5.182
A@5 without 30: 5.175
A@5 without 31: 5.173
A@5 without 32: 5.177
A@5 without 33: 5.185
A@5 without 34: 5.18
A@5 without 35: 5.18
A@5 without 36: 5.174
A@5 without 37: 5.169
A@5 without 38: 5.177
A@5 without 39: 5.175
A@5 without 40: 5.165
A@5 without 41: 5.173
A@5 without 42: 5.176
A@5 without 43: 5.17
A@5 without 44: 5.172
A@5 without 45: 5.173
A@5 without 46: 5.176
A@5 without 47: 5.175
A@5 wi

In [40]:
extra_columns.append("34")
extra_columns.append("35")
extra_columns.append("18")
extra_columns.append("19")

In [41]:
extra_columns

['57', '34', '35', '18', '19']

In [42]:
%%time
max_score = 0;
max_score_field = "";

original_column_list = rscaler_base.columns.copy().to_list()


for k in extra_columns:
    if k in original_column_list:
        original_column_list.remove(k)

for i in original_column_list:
    temp_columns = original_column_list.copy()
    temp_columns.remove(i)

    train_columns = temp_columns.copy()
    train_columns.append("Target")
    score = buildA5ScoreV2(rscaler_base[temp_columns], rscaler_train[train_columns])
    print(f"A@5 without {i}:", score)
    if score > max_score:
        max_score = score
        max_score_field = i

A@5 without 0: 5.169
A@5 without 1: 5.165
A@5 without 2: 5.161
A@5 without 3: 5.165
A@5 without 4: 5.154
A@5 without 5: 5.165
A@5 without 6: 5.144
A@5 without 7: 5.156
A@5 without 8: 5.165
A@5 without 9: 5.173
A@5 without 10: 5.169
A@5 without 11: 5.164
A@5 without 12: 5.17
A@5 without 13: 5.168
A@5 without 14: 5.176
A@5 without 15: 5.169
A@5 without 16: 5.169
A@5 without 17: 5.166
A@5 without 20: 5.164
A@5 without 22: 5.166
A@5 without 23: 5.171
A@5 without 24: 5.176
A@5 without 26: 5.169
A@5 without 27: 5.166
A@5 without 28: 5.163
A@5 without 29: 5.16
A@5 without 30: 5.162
A@5 without 31: 5.168
A@5 without 32: 5.175
A@5 without 33: 5.175
A@5 without 36: 5.167
A@5 without 37: 5.166
A@5 without 38: 5.159
A@5 without 39: 5.155
A@5 without 40: 5.159
A@5 without 41: 5.161
A@5 without 42: 5.168
A@5 without 43: 5.171
A@5 without 44: 5.165
A@5 without 45: 5.176
A@5 without 46: 5.168
A@5 without 47: 5.172
A@5 without 48: 5.17
A@5 without 49: 5.171
A@5 without 50: 5.17
A@5 without 51: 5.175
A@

In [43]:
%%time
original_column_list = rscaler_base.columns.copy().to_list()


for k in extra_columns:
    if k in original_column_list:
        original_column_list.remove(k)

train_columns = temp_columns.copy()
train_columns.append("Target")
score = buildA5ScoreV2(rscaler_base[temp_columns], rscaler_train[train_columns], False, 1000)
print(f"A@5 without {i}:", score)

A@5 without 71: 85.291
CPU times: total: 50min 39s
Wall time: 2min 45s


In [44]:
%%time
original_column_list = rscaler_base.columns.copy().to_list()

score_original = buildA5ScoreV2(rscaler_base, rscaler_train, False, 1500)
print(f"A@5 original:", score)

A@5 original: 85.291
CPU times: total: 51min 36s
Wall time: 3min 3s
